In [59]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import ast
import matplotlib.pyplot as plt
import cv2
from glob import glob
import tensorflow as tf

In [60]:
fnames = glob('/Users/erictang/Desktop/Github/DoogleDrawing/train_simplified/*.csv')
cnames = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word','class_id']
drawlist = []
files_number = 2
images_number = 3
for i in range(files_number):
    f = fnames[i]
    first = pd.read_csv(f, nrows=images_number)
    first = first[first.recognized==True]
    first[6] = i
    drawlist.append(first)
draw_df = pd.DataFrame(np.concatenate(drawlist),columns = cnames)


In [61]:
def draw2img(drawing, shape =(32,32), lw = 4):
    fig, ax = plt.subplots()
    for x,y in drawing:
        ax.plot(x, y,'g',  marker='.', linewidth = lw) #  marker='.', See which line width is better, <4 might be good
    ax.axis('off')
    fig.canvas.draw()    
    X = np.array(fig.canvas.renderer._renderer)
    plt.close(fig)
    temp = (cv2.resize(X, shape) / 255.)[::-1]
    return temp[:,:,1] # only green channel, as we have drawn with green

In [62]:
draw_df_number = len(draw_df)
drawings = [ast.literal_eval(pts) for pts in draw_df[0:draw_df_number]['drawing'].values] 
images = []
for d in drawings:
        image = draw2img(d, (32,32),4)
        images.append(image)
x_train = np.array(images)
classes = draw_df.class_id.tolist()
y_train = np.array(classes)


In [63]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(340, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=train_size)

Epoch 1/6
6/6 [==============================] - 1s 172ms/step - loss: 6.3637 - acc: 0.0000e+00
Epoch 2/6
6/6 [==============================] - 0s 5ms/step - loss: 1.2055 - acc: 0.5000
Epoch 3/6
6/6 [==============================] - 0s 5ms/step - loss: 1.5427 - acc: 0.5000
Epoch 4/6
6/6 [==============================] - 0s 7ms/step - loss: 0.7114 - acc: 0.6667
Epoch 5/6
6/6 [==============================] - 0s 6ms/step - loss: 1.1548 - acc: 0.5000
Epoch 6/6
6/6 [==============================] - 0s 6ms/step - loss: 0.7419 - acc: 0.5000
